# 📄 Explore Raw Documents

Цей ноутбук демонструє, які документи знаходяться у папці `data/raw/`.
Використовується для перевірки бази знань перед індексацією RAG.

In [ ]:
from pathlib import Path

RAW_DIR = Path("../data/raw")

print("📂 Files in data/raw:\n")
for p in RAW_DIR.glob("*"):
    print("=" * 40)
    print(f"File: {p.name}")
    try:
        text = p.read_text(encoding="utf-8")
        print(text[:500])
    except Exception as e:
        print("❌ Cannot read file:", e)